In [6]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests

AMAZON_URL = "https://www.amazon.in/"

AMAZON_CLOTHING_PAGE_URL_1 = "https://www.amazon.in/s?i=apparel&bbn=1968024031&rh=n%3A1571271031%2Cn%3A1968024031%2Cn%3A1968120031%2Cn%3A94998646031%2Cp_85%3A10440599031%2Cp_n_feature_browse-bin%3A95166419031%2Cp_n_pct-off-with-tax%3A2665401031%2Cp_n_size_six_browse-vebin%3A11969821031%7C11969823031%2Cp_89%3AAllen+Solly%7CAmazon+Brand+-+Symbol%7CArrow%7CCalvin+Klein%7CColumbia%7CJack+%26+Jones%7CLacoste%7CLee%7CLevi%27s%7CPark+Avenue%7CPepe+Jeans%7CPeter+England%7CPuma%7CReebok%7CSpykar%7CThe+Indian+Garage+Co%7CThe+Souled+Store%7CTommy+Hilfiger%7CU.S.+POLO+ASSN.%7CU.S.+Polo+Assn%7CUNITED+COLORS+OF+BENETTON%7CVan+Heusen%7CWrangler&s=popularity-rank&dc&page=2&hidden-keywords=-formal-ethnic-sport-women-boy-girl-thermal-innerwear-scarves-glass-frame-sleep-sunglasses-swim-suit-rain-accessories-winter-jacket-sweater-kurta-kurti-sleep-mask-pyjama-fabric-glove-belt-suspender-suit-short-trouser-blazer-dhoti-coat-sock-utility-jeans-suit-blazer-winter&_encoding=UTF8%2CUTF8&pf_rd_i=1968024031&pf_rd_i=1968024031%2C27059570031%2C27059570031%2C27059570031%2C27059570031&pf_rd_m=A1K21FY43GMZF8%2CA1VBAL9TL5WCBF%2CA1VBAL9TL5WCBF%2CA1VBAL9TL5WCBF%2CA1VBAL9TL5WCBF&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_p=41702215-f8bb-4a32-bc91-ebdaef2c07e0&pf_rd_p=Preview-Of-3bd7aeb3-d9e4-4d61-9677-a90e905dfd9d&pf_rd_r=8AQ59E9PY177SK641PJC&pf_rd_r=TSV9R0RZJ2QDN7G60VRM&pf_rd_s=merchandised-search-7&pf_rd_s=merchandised-search-8%2Cmerchandised-search-8%2Cmobile-hybrid-8%2Cmobile-hybrid-12_CW%2Cmobile-hybrid-13_CW&pf_rd_t=101%2C30901%2C30901%2C30901&qid=1720188190&rnid=3837712031&ref=sr_pg_2"

# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
#     "Accept-Language": "en-US, en;q=0.5",
# }

HEADERS_1 = {"User-Agent": "hihwihiw  iwhd oiwhiwhiw h 78 798 j kq", "Accept-Language": "en-US, en;q=0.5"}
HEADERS_2 = {"User-Agent": "nkuy8 i89n ihi hiu whuig 7y huiw ih#%#5 r6 4^", "Accept-Language": "en-US, en;q=0.5"}

cloths_data_dict = {
    "Product Name": [],
    "Item model number": [],
    "ASIN": [],
    "Department": [],
    "Date First Available": [],
    "Product Dimensions": [],
    "Sizes Available": [],
    "Colours Available": [],
    "Collar style": [],
    "Fit type": [],
    "Length": [],
    "Material composition": [],
    "Country of Origin": [],
    "Manufacturer": [],
    "Packer": [],
    "Importer": [],
    "Item Weight": [],
    "Discount (in %)": [],
    "Price (in Rs.)": [],
    "Ratings": [],
    "Product Description": [],
}

In [7]:
#### Smartphones ####

def get_product_name(product_page):

    title_string = ""

    try:

        # Outer Tag Object
        product_name = product_page.find(
            name="span",
            attrs={
                "id": "productTitle",
                "class": "a-size-large product-title-word-break",
            },
        )

        # Inner NavigatableString Object
        title_value = product_name.text

        # Product Name as a string value
        title_string = title_value.strip()

    except AttributeError as error:
        title_string = np.nan

    return title_string if title_string != "" else np.nan


def get_product_sizes_available(product_page):

    sizes_available = ""

    try:

        # Outer Tag Object
        product_price_dropdown = product_page.find(
            name="select", attrs={"id": "native_dropdown_selected_size_name"}
        )

        for option in product_price_dropdown.find_all(name="option"):
            size_txt = option.text.strip()
            if size_txt != "Select":
                if sizes_available == "":
                    sizes_available += size_txt
                else:
                    sizes_available += ", " + size_txt

    except AttributeError as error:
        sizes_available = np.nan

    return sizes_available if sizes_available != "" else np.nan


def get_product_colours_available(product_page):

    colours_available = ""

    try:

        # Outer Tag Object
        product_colors_imgs = product_page.find_all(
            name="img", attrs={"class": "imgSwatch"}
        )

        for img_tag in product_colors_imgs:
            color_txt = img_tag.get('alt')
            if colours_available == "":
                colours_available += color_txt
            else:
                colours_available += ", " + color_txt

    except AttributeError as error:
        colours_available = np.nan

    return colours_available if colours_available != "" else np.nan

def get_product_discount_in_percentage(product_page):

    discount = ""

    try:

        # Outer Tag Object
        product_discount = product_page.find(
            name="span",
            attrs={
                "class": "a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage"
            },
        )

        # Inner NavigatableString Object
        discount_value = product_discount.text

        # Product Price as a int value
        discount = discount_value.strip().replace("-", "").replace("%", "")

        discount = int(discount)

    except AttributeError as error:
        discount = 0

    return discount


def get_product_price_in_rs(product_page):

    price = ""

    try:

        # Outer Tag Object
        product_price = product_page.find(name="span", attrs={"class": "a-price-whole"})

        # Inner NavigatableString Object
        price_value = product_price.text

        # Product Price as a int value
        price = price_value.strip().replace(",", "").replace(".", "")

        price = int(price)

    except AttributeError as error:
        price = 0

    return price

def get_product_details_1(product_page):

    details = {}
    key = []
    value = []

    try:

        # Outer Tag Object
        product_details = product_page.find(
            name="div",
            attrs={
                "class": "a-expander-content a-expander-partial-collapse-content",
                "data-expanded": "false",
            },
        )

        keys_divs = product_details.find_all(
            name="div",
            attrs={
                "class": "a-fixed-left-grid-col a-col-left",
            },
        )

        values_divs = product_details.find_all(
            name="div",
            attrs={
                "class": "a-fixed-left-grid-col a-col-right",
            },
        )

        for key_div, value_div in zip(keys_divs, values_divs):
            key_txt = key_div.find(
                name="span",
                attrs={
                    "class": "a-color-base",
                }
            ).text.strip()

            value_txt = value_div.find(
                name="span",
                attrs={
                    "class": "a-color-base",
                }
            ).text.strip()

            key.append(key_txt)
            value.append(value_txt)
            details = dict(zip(key, value))

        temp_dict = details.copy()
        details.clear()

        for key, value in temp_dict.items():
            if key in [
                "Collar style",
                "Neck style",
                "Fit type",
                "Style",
                "Length",
                "Material composition",
                "Country of Origin",
            ]:
                if key == "Neck style":
                    key = "Collar style"
                if key == "Style":
                    key = "Fit type"
                details[key] = value

    except AttributeError:
        return details

    return details


def get_product_details_2(product_page):

    details = {}
    key = []
    value = []

    try:

        # Outer Tag Object
        product_details_div = product_page.find(
            name="div",
            attrs={
                "id": "detailBullets_feature_div",
            },
        )

        keys_spans = product_details_div.find_all(
            name="span",
            attrs={
                "class": "a-text-bold",
            },
        )

        values_spans = product_details_div.find_all(
            name="span",
            attrs={"class": None}
        )

        for key_span, value_span in zip(keys_spans, values_spans):
            key_txt = (
                key_span.text.replace(":", "")
                .replace("\u200f", "")
                .replace("\n", "")
                .replace("\u200e", "")
                .strip()
            )
            value_txt = (
                value_span.text
                .encode("ascii", "ignore")
                .decode("utf-8")
            )
            key.append(key_txt)
            value.append(value_txt)
            details = dict(zip(key, value))

        temp_dict = details.copy()
        details.clear()

        for key, value in temp_dict.items():
            if key in [
                "Item model number",
                "ASIN",
                "Department",
                "Date First Available",
                "Product Dimensions",
                "Manufacturer",
                "Packer",
                "Importer",
                "Item Weight",
            ]:
                details[key] = value

    except AttributeError:
        return details

    return details

def get_product_ratings(product_page):

    ratings_string = ""

    try:

        # Outer Tag Object
        product_name = product_page.find(
            name="span",
            attrs={
                "data-hook": "rating-out-of-text",
            },
        )

        # Inner NavigatableString Object
        rating_value = product_name.text

        # Product Name as a string value
        ratings_string = rating_value.strip() + " stars."

    except AttributeError as error:
        ratings_string = np.nan

    return ratings_string if ratings_string != "" else np.nan

def get_product_description(product_page):

    product_description = ""

    try:

        # Outer Tag Object
        desc_div = product_page.find(
            name="div",
            attrs={
                "id": "productDescription",
                "class": "a-section a-spacing-small",
            },
        )

        # Inner NavigatableString Object
        desc_txt = desc_div.find(name="span").text

        # Product Name as a string value
        product_description = desc_txt.strip()

    except AttributeError as error:
        product_description = np.nan

    return product_description if product_description != "" else np.nan

for i in range(1):

    URL = AMAZON_CLOTHING_PAGE_URL_1

    amazon_url_response = requests.get(
        url=AMAZON_CLOTHING_PAGE_URL_1, headers=HEADERS_1
    )

    amazon_parsed_html = BeautifulSoup(amazon_url_response.content, "html.parser")

    print("Status Code Of Response: " + str(amazon_url_response.status_code))

    # Scrap next page url
    choths_page_next_url = amazon_parsed_html.find(
        name="a",
        attrs={"class": "s-pagination-item s-pagination-button"},
    )

    product_urls_anchor_tags = amazon_parsed_html.find_all(
        name="a",
        attrs={
            "class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"
        },
    )

    for product in product_urls_anchor_tags:
        product_relative_url = product.get("href")
        product_absolute_url = AMAZON_URL + product_relative_url

        url_response = requests.get(url=product_absolute_url, headers=HEADERS_2)

        parsed_html_page = BeautifulSoup(url_response.content, "html.parser")

        # print(get_product_name(parsed_html_page))
        # print(get_product_price_in_rs(parsed_html_page))
        # print(get_product_discount_in_percentage(parsed_html_page))
        # print(get_product_sizes_available(parsed_html_page))
        # print(get_product_colours_available(parsed_html_page))
        # print(get_product_ratings(parsed_html_page))
        # print(get_product_description(parsed_html_page))
        # print(get_product_details_1(parsed_html_page))
        # print(get_product_details_2(parsed_html_page))

        cloths_data_dict["Product Name"].append(get_product_name(parsed_html_page))
        detail_1 = get_product_details_1(parsed_html_page)

        # "Collar style",
        # "Fit type",
        # "Length",
        # "Material Composition",
        # "Country of Origin"
        cloths_data_dict["Collar style"].append(
            detail_1["Collar style"] if "Collar style" in detail_1 else np.nan
        )
        cloths_data_dict["Fit type"].append(
            detail_1["Fit type"] if "Fit type" in detail_1 else np.nan
        )
        cloths_data_dict["Length"].append(
            detail_1["Length"] if "Length" in detail_1 else np.nan
        )
        cloths_data_dict["Material composition"].append(
            detail_1["Material composition"] if "Material composition" in detail_1 else np.nan
        )
        cloths_data_dict["Country of Origin"].append(
            detail_1["Country of Origin"] if "Country of Origin" in detail_1 else np.nan
        )

        detail_2 = get_product_details_2(parsed_html_page)
        # "Item model number",
        # "ASIN",
        # "Department",
        # "Date First Available",
        # "Product Dimensions",
        # "Manufacturer",
        # "Packer",
        # "Importer",
        # "Item Weight",
        cloths_data_dict["Item model number"].append(
            detail_2["Item model number"] if "Item model number" in detail_2 else np.nan
        )
        cloths_data_dict["ASIN"].append(
            detail_2["ASIN"] if "ASIN" in detail_2 else np.nan
        )
        cloths_data_dict["Department"].append(
            detail_2["Department"] if "Department" in detail_2 else np.nan
        )
        cloths_data_dict["Date First Available"].append(
            detail_2["Date First Available"] if "Date First Available" in detail_2 else np.nan
        )
        cloths_data_dict["Product Dimensions"].append(
            detail_2["Product Dimensions"] if "Product Dimensions" in detail_2 else np.nan
        )
        cloths_data_dict["Manufacturer"].append(
            detail_2["Manufacturer"] if "Manufacturer" in detail_2 else np.nan
        )
        cloths_data_dict["Packer"].append(
            detail_2["Packer"] if "Packer" in detail_2 else np.nan
        )
        cloths_data_dict["Importer"].append(
            detail_2["Importer"] if "Importer" in detail_2 else np.nan
        )
        cloths_data_dict["Item Weight"].append(
            detail_2["Item Weight"] if "Item Weight" in detail_2 else np.nan
        )

        cloths_data_dict["Sizes Available"].append(get_product_sizes_available(parsed_html_page))
        cloths_data_dict["Colours Available"].append(get_product_colours_available(parsed_html_page))
        cloths_data_dict["Discount (in %)"].append(get_product_discount_in_percentage(parsed_html_page))
        cloths_data_dict["Price (in Rs.)"].append(get_product_price_in_rs(parsed_html_page))
        cloths_data_dict["Ratings"].append(get_product_ratings(parsed_html_page))
        cloths_data_dict["Product Description"].append(get_product_description(parsed_html_page))

    URL = choths_page_next_url

amazon_clothing_product_data = pd.DataFrame(cloths_data_dict)
amazon_clothing_product_data.to_csv("amazon_clothing_product_data_21.csv")
amazon_clothing_product_data

Status Code Of Response: 200


,Product Name,Item model number,ASIN,Department,Date First Available,Product Dimensions,Sizes Available,Colours Available,Collar style,Fit type,...,Material composition,Country of Origin,Manufacturer,Packer,Importer,Item Weight,Discount (in %),Price (in Rs.),Ratings,Product Description
0,Peter England Men's Regular Fit Shirt,PCSFLSLFP98244,B0CRHQ8WFK,Men,4 January 2024,27.5 x 22.5 x 2 cm; 300 g,"38, 39, 40, 42, 44, 46","Green, white",Dom,Slim Fit,...,100% Cotton,India,"ABFRL, Aditya Birla Fashion and Retail Ltd,7th...","Aditya Birla Fashion and Retail Ltd,7th Floor,...",NaN,300 g,51,989,3.4 out of 5 stars.,Peter England Casual Shirt
1,Arrow Plaid Check Twill Casual Shirt,ASADOSH01862,B0BTQ4XVTB,Men,2 February 2023,31 x 2 x 3 cm; 200 g,"39, 40, 42","Green, Red",Spread Collar,Slim Fit,...,100% Cotton,India,"M/S. Girish Exports, Somanahalli Industrial Ar...","M/S. Girish Exports, Somanahalli Industrial Ar...","M/S. Girish Exports, Somanahalli Industrial Ar...",200 g,50,1250,3.8 out of 5 stars.,Arrow's plaid cotton shirt is a stylish one-st...
2,Van Heusen Men Slim Fit Black Shirt 40,NaN,B0BNHNG7MX,Men,28 November 2022,NaN,NaN,NaN,Dom,Slim Fit,...,NaN,India,Aditya Birla Fashion And Retail Limited,NaN,NaN,300 g,51,1469,NaN,Upgrade your wardrobe with this Black Print sh...
3,Van Heusen Men's Solid Slim Fit Casual Shirt,VDSFESKFW35327,B07NR1LB1H,Men,14 February 2019,28 x 27 x 2 cm; 350 g,"39, 40",NaN,Button Front,Slim Fit,...,Cotton,India,"V Dot by Van Heusen, Aditya Birla Fashion and ...","Aditya Birla Fashion and Retail,r 288/2, Build...",NaN,350 g,51,1071,3.6 out of 5 stars.,This smart casual shirt is perfect for all cas...
4,Park Avenue Men's Slim Fit Solid Pattern Polyv...,PCSA02921-K9,B0D47GRB65,Men,15 May 2024,NaN,"39, 42, 44",NaN,Cut Away Collar,Slim Fit,...,Polyviscose Blend,India,"Punit Creation (Bangalore) Urban 44, Near Maca...",Eshopbox E-Commerce Private Limited,NaN,280 g,55,1079,NaN,Elevate your casual look with our City chic ca...
5,Park Avenue Men's Slim Fit Checkered Pattern P...,PCSA02805-F5,B0D37XCPTN,Men,3 May 2024,NaN,NaN,NaN,Cutaway Collar,Slim Fit,...,Pure Cotton,India,"New Bury Fashions,Plot No.10, 11, 12C, Gidc, U...",Eshopbox E-Commerce Private Limited,NaN,280 g,55,989,NaN,Elevate your casual look with our City chic ca...
6,Park Avenue Men's Solid Pattern Slim Fit Pure ...,PCSA02813-B2,B0D2XLQS6T,Men,24 April 2024,NaN,"39, 40, 42, 44","Light Blue, Medium Violet, Light Yellow",Cutaway,Slim Fit,...,Pure Cotton,India,"Radhe Krishna Clothings Private Limited, Berat...",Eshopbox E-Commerce Private Limited,NaN,280 g,55,1079,NaN,Elevate your casual look with our City chic ca...
7,Park Avenue Men's Slim Fit Solid Pattern Pure ...,PCSA02813-W1,B0D47HVRWY,Men,15 May 2024,NaN,"39, 40, 42, 44",NaN,Cutaway,Slim Fit,...,Pure Cotton,India,"Radhe Krishna Clothings Private Limited, Berat...",Eshopbox E-Commerce Private Limited,NaN,280 g,55,1079,NaN,Elevate your casual look with our City chic ca...
8,Park Avenue Slim Fit Medium Red Casual Shirt f...,PCSA02603-R4,B0BY977YDK,Men,13 March 2023,NaN,NaN,NaN,Cutaway,Slim Fit,...,Cotton,India,"NEW BURY FASHIONS, Ace Clothing Company Pvt. L...",Raymond Limited,NaN,350 g,55,944,NaN,F/S CUTWAY CLLR BONE POCKET SLM FT
9,Jack & Jones Men's Striped Slim Fit Button-Dow...,12207556-Rhubarb,B09HCKLL8K,Men,28 September 2021,75.9 x 103.6 x 75.9 cm; 400 g,"39, 40, 42, 44, 46",NaN,Dom,Slim Fit,...,100% Cotton,India,"Aquarelle India Pvt Ltd, Aquarelle India Pvt L...","Best United India comforts PVT.LTD, The Lalit ...","Best United India comforts PVT.LTD, The Lalit ...",400 g,70,1050,NaN,Jack & Jones Men Casual Wear Cotton Striped Bl...
